In [130]:
import urllib
import lxml
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [296]:
url1 = 'https://en.wikipedia.org/wiki/Main_Page'
url2 = 'https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc'
url3 = 'https://www.imdb.com/india/top-rated-indian-movies/'
url4 = 'https://meesho.com/bags-ladies/pl/p7vbp'

In [132]:
def question1(url):
    text = urllib.request.urlopen(url).read().decode('utf-8')
    soup = bs(text)
    headers = soup.find_all('head')
    return headers

In [203]:
def question2(url):
    '''Since the top 100 are split over the two pages, two page links are created'''
    page1 = requests.get(url)
    page2 = requests.get(url + '&start=51&ref_=adv_nxt')
    movie_list = []
    for _ in [page1, page2]:
        soup = bs(_.content)
        text = soup.find_all('h3', {'class':'lister-item-header'})
        movie_list += [_.text.split('\n')[1:4] for _ in text]
    df = pd.DataFrame(movie_list, columns = ['Rank', 'Movie Name', 'Year'])
    df.set_index('Rank', inplace = True)
    return df

In [295]:
def question3(url):
    '''
    Since the imdb site list the top 250 movies, the list is truncated to 
    the top 100 as required
    '''
    page = requests.get(url)
    soup = bs(page.content)
    text = soup.find_all('td', {'class':'titleColumn'})
    rate = soup.find_all('td', {'class':'ratingColumn imdbRating'})
    names  = [_.text.split('\n')[1:4] for _ in text][0:100] 
    rating = [_.text.split('\n')[1] for _ in rate][0:100]

    df = pd.DataFrame(names, columns = ['Rank', 'Movie Name', 'Year'])
    df.set_index('Rank', inplace = True)
    df['rating'] = rating
    return df

In [356]:
def question4(url):
    page = requests.get(url)
    text = soup(page.content)
    name = soup.find_all('p', {'class':'Text__StyledText-sc-oo0kvp-0 bWSOET NewProductCard__ProductTitle_Desktop-sc-j0e7tu-4 cQhePS NewProductCard__ProductTitle_Desktop-sc-j0e7tu-4 cQhePS'})
    price= soup.find_all('h5',{'class':'Text__StyledText-sc-oo0kvp-0 hiHdyy'})
    disc = soup.find_all('span', {'class' : 'Text__StyledText-sc-oo0kvp-0 lnonyH'})
    name = [_.text for _ in name]
    price= [_.text for _ in price]
    disc = [_.text for _ in disc]
    # to accomodate for cases with no discount
    p = re.compile('₹0')
    i = next(i for i in range(len(price)) if p.findall(price[i]))
    disc.insert(i, '0% off')
    df = pd.DataFrame(data = {'Name':name, 'Price':price, 'Discount':disc})
    return df
    

,Name,Price,Discount
0,Elegant Fashionable Women Handbags,₹192,30% off
1,Gorgeous Attractive Women Handbags,₹342,23% off
2,Classic Fancy Women Handbags,₹207,30% off
3,Classic Versatile Women Handbags,₹0 onwards,0% off
4,Gorgeous Versatile Women Handbags,₹234,30% off
5,Trendy Fancy Women Handbags,₹464,18% off
6,Voguish Classy Women Handbags,₹187,30% off
7,Elegant Versatile Women Handbags,₹234,30% off
8,Ravishing Stylish Women Handbags,₹227,30% off
9,Ravishing Alluring Women Handbags,₹282,26% off


3

In [354]:
disc.insert(i, '0% off')

In [355]:
df = pd.DataFrame(data = {'Name':name, 'Price':price, 'Discount':disc})

ValueError: All arrays must be of the same length

In [353]:
df

,Name,Price,Discount
0,Elegant Fashionable Women Handbags,₹192,0% off
1,Gorgeous Attractive Women Handbags,₹342,30% off
2,Classic Fancy Women Handbags,₹207,23% off
3,Classic Versatile Women Handbags,₹0 onwards,30% off
4,Gorgeous Versatile Women Handbags,₹234,30% off
5,Trendy Fancy Women Handbags,₹464,18% off
6,Voguish Classy Women Handbags,₹187,30% off
7,Elegant Versatile Women Handbags,₹234,30% off
8,Ravishing Stylish Women Handbags,₹227,30% off
9,Ravishing Alluring Women Handbags,₹282,26% off


In [329]:
i = next((_ for _ in price if _ == 0), None)

In [339]:
price

['₹192',
 '₹342',
 '₹207',
 '₹0 onwards',
 '₹234',
 '₹464',
 '₹187',
 '₹234',
 '₹227',
 '₹282',
 '₹214',
 '₹171',
 '₹233',
 '₹137',
 '₹274',
 '₹234',
 '₹200',
 '₹206',
 '₹232',
 '₹210']